In [2]:
import numpy as np
from numpy import array
import mpmath as mp
from sympy import Rational, I, Matrix, conjugate
from lib.Quantum_Gate import*

In [3]:
CNOT = QuantumGate(n = 2, free_bits_dict={}, target_bits_dict={1: X}, controlled_bits_dict={0: 1})

In [4]:
(QuantumGate(n = 4, free_bits_dict={}, target_bits_dict={1: X}, controlled_bits_dict={2: 0, 3: 1}).compute_matrix_rep() - QuantumGate_old(n = 4, free_bits_dict={}, target_bits_dict={1: X}, controlled_bits_dict={2: 0, 3: 1}).matrix_rep)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [5]:
(QuantumGate(n = 4, free_bits_dict={}, target_bits_dict={2*1+1: X}, controlled_bits_dict={2*1: 1}).compute_matrix_rep() == QuantumGate_old(n = 4, free_bits_dict={}, target_bits_dict={2*1+1: X}, controlled_bits_dict={2*1: 1}).matrix_rep).all()

True

In [6]:
CNOT.matrix_rep

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

In [7]:
[
    (CNOT.matrix_rep @ kron(X, Id2) @ CNOT.matrix_rep == kron(X, Id2) @ kron(Id2, X)).all(),
    (CNOT.matrix_rep @ kron(Y, Id2) @ CNOT.matrix_rep == kron(Y, Id2) @ kron(Id2, X)).all(),
    (CNOT.matrix_rep @ kron(Z, Id2) @ CNOT.matrix_rep == kron(Z, Id2)).all(),
    (CNOT.matrix_rep @ kron(Id2, X) @ CNOT.matrix_rep == kron(Id2, X)).all(),
    (CNOT.matrix_rep @ kron(Id2, Y) @ CNOT.matrix_rep == kron(Z, Id2) @ kron(Id2, Y)).all(),
    (CNOT.matrix_rep @ kron(Id2, Z) @ CNOT.matrix_rep == kron(Z, Id2) @ kron(Id2, Z)).all(),
]

[True, True, True, True, True, True]

In [8]:
Toffoli_c0c1_t2 = QuantumGate(n = 3, free_bits_dict={}, target_bits_dict={2: X}, controlled_bits_dict={0: 1, 1: 1})
Toffoli_c0c2_t1 = QuantumGate(n = 3, free_bits_dict={}, target_bits_dict={1: X}, controlled_bits_dict={0: 1, 2: 1})
Toffoli_c1c2_t0 = QuantumGate(n = 3, free_bits_dict={}, target_bits_dict={0: X}, controlled_bits_dict={1: 1, 2: 1})

In [9]:
Fredkin = array([[1, 0, 0, 0, 0, 0, 0, 0], 
                 [0, 1, 0, 0, 0, 0, 0, 0], 
                 [0, 0, 1, 0, 0, 0, 0, 0],
                 [0, 0, 0, 1, 0, 0, 0, 0],
                 [0, 0, 0, 0, 1, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 1, 0],
                 [0, 0, 0, 0, 0, 1, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 1]])
                  
all_combo = list(itertools.product(*[[
                                      Toffoli_c0c1_t2.matrix_rep, 
                                      Toffoli_c0c2_t1.matrix_rep, 
                                      Toffoli_c1c2_t0.matrix_rep]
                                    ] * 3))
                                    
eval_combo = list(map(lambda comb: (reduce(lambda m1, m2: m1 @ m2, comb) == Fredkin).all(), all_combo))
print(eval_combo)

[False, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]


In [10]:
[
    (Toffoli_c0c1_t2.matrix_rep @ Toffoli_c0c2_t1.matrix_rep @ Toffoli_c0c1_t2.matrix_rep == Fredkin).all(),
    (Toffoli_c0c2_t1.matrix_rep @ Toffoli_c0c1_t2.matrix_rep @ Toffoli_c0c2_t1.matrix_rep == Fredkin).all()
]

[True, True]

In [11]:
G1 = QuantumGate(n = 3, free_bits_dict={0: H}, target_bits_dict={}, controlled_bits_dict={})
G2 = QuantumGate(n = 3, free_bits_dict={}, target_bits_dict={0: S}, controlled_bits_dict={1: 1})
G3 = QuantumGate(n = 3, free_bits_dict={}, target_bits_dict={0: T}, controlled_bits_dict={2: 1})
G4 = QuantumGate(n = 3, free_bits_dict={1: H}, target_bits_dict={}, controlled_bits_dict={})
G5 = QuantumGate(n = 3, free_bits_dict={}, target_bits_dict={1: S}, controlled_bits_dict={2: 1})
G6 = QuantumGate(n = 3, free_bits_dict={2: H}, target_bits_dict={}, controlled_bits_dict={})


In [12]:
circuit = [G1, G2, G3, G4, G5, G6]
circuit = map(lambda gate: gate.matrix_rep, reversed(circuit))
swap = reduce(lambda m1, m2: m1 + m2, [(BasisKet(s1, s2, s3).matrix_rep @ BasisKet(s3, s2, s1).matrix_rep.T) for s1 in range(2) for s2 in range(2) for s3 in range(2)])
FT = sqrt(8) * swap @ reduce(lambda g1, g2: g1 @ g2, circuit)

In [23]:
FT.round(3)

array([[ 1.   +0.j   ,  1.   +0.j   ,  1.   +0.j   ,  1.   +0.j   ,
         1.   +0.j   ,  1.   +0.j   ,  1.   +0.j   ,  1.   +0.j   ],
       [ 1.   +0.j   ,  0.707+0.707j,  0.   +1.j   , -0.707+0.707j,
        -1.   +0.j   , -0.707-0.707j,  0.   -1.j   ,  0.707-0.707j],
       [ 1.   +0.j   ,  0.   +1.j   , -1.   +0.j   ,  0.   -1.j   ,
         1.   +0.j   ,  0.   +1.j   , -1.   +0.j   ,  0.   -1.j   ],
       [ 1.   +0.j   , -0.707+0.707j,  0.   -1.j   ,  0.707+0.707j,
        -1.   +0.j   ,  0.707-0.707j,  0.   +1.j   , -0.707-0.707j],
       [ 1.   +0.j   , -1.   +0.j   ,  1.   +0.j   , -1.   +0.j   ,
         1.   +0.j   , -1.   +0.j   ,  1.   +0.j   , -1.   +0.j   ],
       [ 1.   +0.j   , -0.707-0.707j,  0.   +1.j   ,  0.707-0.707j,
        -1.   +0.j   ,  0.707+0.707j,  0.   -1.j   , -0.707+0.707j],
       [ 1.   +0.j   ,  0.   -1.j   , -1.   +0.j   ,  0.   +1.j   ,
         1.   +0.j   ,  0.   -1.j   , -1.   +0.j   ,  0.   +1.j   ],
       [ 1.   +0.j   ,  0.707-0.707j,  0.

In [14]:
omega = exp(I * pi / 4)
FT_res = [[1, 1, 1, 1, 1, 1, 1, 1]]
FT_res += ([[omega**p for p in range(0, i * 8, i)] for i in range(1, 8)])
FT_res = array(FT_res)
((FT_res - FT).round(10) == 0).all()

True